## Extra Trees Hyperparameter Tuning (F1-Score)

In [4]:
# Load Libraries

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score

from sklearn.ensemble import ExtraTreesClassifier

import optuna

In [5]:
# Load Data

df = pd.read_csv('bank_4.csv', index_col=0)

In [6]:
# Train / Test Split

X = df.drop(columns=['churn', 'complain', 'umap_1', 'umap_2'])
y = df['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [10]:
# Hyperparameter tuning

def objective(trial):
    
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
        'max_features': trial.suggest_int('max_features', 4, 53),
        'max_depth': trial.suggest_int('max_depth', 2, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 32),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 32)
    }

    model = ExtraTreesClassifier(
        **params,
        random_state=42,
        n_jobs=-1)
        
    threshold = 0.32
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    scores = []
    
    for tr, te in skf.split(X_train, y_train):
        
        X_tr, X_te = X_train.iloc[tr], X_train.iloc[te]
        y_tr, y_te = y_train.iloc[tr], y_train.iloc[te]
        
        model.fit(X_tr, y_tr)
        prob = model.predict_proba(X_te)[:, 1]
        y_pred = np.where(prob < threshold, 0, 1)
        
        scores.append(f1_score(y_te, y_pred))
        
    return np.mean(scores)

In [12]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, show_progress_bar=True)

[I 2024-07-09 22:23:08,689] A new study created in memory with name: no-name-ba1620d4-887f-4ad9-83df-a7513bc58aae


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2024-07-09 22:23:21,295] Trial 0 finished with value: 0.6174258375341066 and parameters: {'n_estimators': 1104, 'max_features': 15, 'max_depth': 39, 'min_samples_split': 32, 'min_samples_leaf': 31}. Best is trial 0 with value: 0.6174258375341066.
[I 2024-07-09 22:23:29,728] Trial 1 finished with value: 0.6063472138917796 and parameters: {'n_estimators': 974, 'max_features': 5, 'max_depth': 46, 'min_samples_split': 2, 'min_samples_leaf': 15}. Best is trial 0 with value: 0.6174258375341066.
[I 2024-07-09 22:23:47,713] Trial 2 finished with value: 0.6175315083852329 and parameters: {'n_estimators': 1554, 'max_features': 17, 'max_depth': 37, 'min_samples_split': 29, 'min_samples_leaf': 30}. Best is trial 2 with value: 0.6175315083852329.
[I 2024-07-09 22:24:04,780] Trial 3 finished with value: 0.6170859601574837 and parameters: {'n_estimators': 1373, 'max_features': 40, 'max_depth': 48, 'min_samples_split': 30, 'min_samples_leaf': 30}. Best is trial 2 with value: 0.6175315083852329.
[I 

In [36]:
print("Best trial:", study.best_trial)
print("Best hyperparameters:", study.best_params)

Best trial: FrozenTrial(number=17, state=1, values=[0.6206468427606204], datetime_start=datetime.datetime(2024, 7, 9, 22, 26, 34, 996376), datetime_complete=datetime.datetime(2024, 7, 9, 22, 26, 54, 858765), params={'n_estimators': 1374, 'max_features': 23, 'max_depth': 21, 'min_samples_split': 23, 'min_samples_leaf': 14}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=2000, log=False, low=100, step=1), 'max_features': IntDistribution(high=53, log=False, low=4, step=1), 'max_depth': IntDistribution(high=50, log=False, low=2, step=1), 'min_samples_split': IntDistribution(high=32, log=False, low=2, step=1), 'min_samples_leaf': IntDistribution(high=32, log=False, low=1, step=1)}, trial_id=17, value=None)
Best hyperparameters: {'n_estimators': 1374, 'max_features': 23, 'max_depth': 21, 'min_samples_split': 23, 'min_samples_leaf': 14}
